# Explique com suas palavras o Bagging

 ## Bagging (Bootstrap Aggregation)

Bagging, ou agregação bootstrap, é uma técnica de aprendizado de máquina que melhora a precisão e a estabilidade de modelos com alta variância, especialmente em modelos como árvores de decisão. O processo de Bagging inicia com a **preparação dos dados**, onde o conjunto de dados é dividido em variáveis de entrada (X) e uma variável alvo (y). Em seguida, são criadas **várias amostras aleatórias** a partir do conjunto original, utilizando uma técnica de amostragem com reposição, o que significa que um mesmo exemplo pode aparecer várias vezes na mesma amostra.

Cada uma dessas amostras é usada para **treinar um modelo base independente** (geralmente uma árvore de decisão, embora outros modelos possam ser usados). Assim, cada modelo realiza suas próprias previsões de forma independente quando novos dados são apresentados. Para problemas de **classificação**, as previsões dos modelos são combinadas por meio de uma **votação da maioria**, enquanto para problemas de **regressão**, é utilizada a **média das previsões**.

O Bagging ajuda a reduzir a variância e suavizar as previsões, tornando o modelo final mais robusto e menos suscetível ao overfitting. Uma implementação comum de Bagging é a **Random Forest**, que consiste em múltiplas árvores de decisão treinadas em diferentes amostras de um conjunto de dados. Em Python, o Bagging pode ser implementado facilmente usando as classes `RandomForestClassifier` ou `BaggingClassifier` da biblioteca `sklearn`.


# Passo a Passo para o Bagging

### 1. Preparação dos Dados
   - **Objetivo**: Definir as variáveis de entrada e a variável alvo.
   - **Ações**:
     - Começar com o conjunto de dados completo.
     - Dividir os dados em variáveis de entrada (X) e a variável alvo (y) que deseja prever.

### 2. Divisão do Conjunto de Dados
   - **Objetivo**: Dividir o conjunto de dados em dados de treinamento e teste.
   - **Ações**:
     - Utilizar uma técnica como a `train_test_split` do `sklearn` para dividir os dados, garantindo que uma parte seja reservada para avaliar o modelo final.

### 3. Gerar Amostras para Treinamento (Bootstrapping)
   - **Objetivo**: Criar várias amostras do conjunto de dados original.
   - **Ações**:
     - Para criar cada amostra, realizamos uma "amostragem com reposição" na qual alguns exemplos podem aparecer várias vezes.
     - Escolhemos o número de amostras, que é determinado pelo parâmetro `n_estimators`. Por exemplo, `n_estimators=10` indica que serão criadas 10 amostras.

### 4. Treinamento de Modelos Base
   - **Objetivo**: Treinar um modelo para cada amostra.
   - **Ações**:
     - Para cada amostra gerada, treine um modelo base independente. O modelo base mais comum é uma árvore de decisão, pois é sensível a melhorias com o Bagging, mas outros modelos podem ser usados.
     - Ajuste cada modelo com a sua amostra correspondente.

### 5. Fazer Previsões
   - **Objetivo**: Obtemos previsões de cada modelo treinado.
   - **Ações**:
     - Para cada novo dado (dados de teste ou novos dados de entrada), fazemos com que cada modelo treinado faça uma previsão independente.

### 6. Combinar as Previsões
   - **Objetivo**: Obter uma previsão final para cada dado.
   - **Ações**:
     - Para problemas de classificação, usamos a votação da maioria: a classe mais comum entre as previsões dos modelos é a resposta final.
     - Para problemas de regressão, usamos a média das previsões dos modelos para obter a previsão final.

### 7. Avaliação do Modelo Bagging
   - **Objetivo**: Verificamos o desempenho do modelo Bagging.
   - **Ações**:
     - Avaliamos o modelo usando métricas apropriadas, como acurácia, precisão, ou erro médio absoluto, dependendo do tipo de problema (classificação ou regressão).
     - Compare o desempenho do modelo Bagging com um modelo único para ver se houve melhoria.

### Exemplo de Bagging com `RandomForest` (Automatizado)
   - A `RandomForest`, que usa árvores de decisão como modelo base, é uma implementação comum de Bagging. Em Python, podemos implementar Bagging de forma simples usando `RandomForestClassifier` para classificação ou `RandomForestRegressor` para regressão, ambos disponíveis na biblioteca `sklearn`.


In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import yfinance as yf

### Importação das Bibliotecas

No início do projeto, importamos várias bibliotecas essenciais para a análise e construção do modelo de Bagging:

- **pandas**: Usado para manipulação e análise de dados. O `pandas` é uma biblioteca poderosa para trabalhar com tabelas e séries temporais, permitindo a leitura, limpeza, e estruturação dos dados financeiros.
- **numpy**: Biblioteca fundamental para cálculos numéricos em Python. O `numpy` facilita operações matemáticas e manipulação de arrays, que são úteis para manipulações mais complexas nos dados.
- **sklearn.ensemble.BaggingRegressor**: O `BaggingRegressor` é uma implementação de Bagging, que permite construir modelos de aprendizado de máquina mais robustos combinando previsões de múltiplos estimadores. Escolhemos essa função para criar um modelo mais robusto usando várias árvores de decisão.
- **sklearn.tree.DecisionTreeRegressor**: Utilizado como estimador base dentro do `BaggingRegressor`. Árvores de decisão são comumente usadas no Bagging porque são modelos instáveis que se beneficiam da agregação, melhorando sua precisão.
- **sklearn.model_selection.train_test_split**: Essa função divide os dados em conjuntos de treino e teste, permitindo avaliar o desempenho do modelo em dados não vistos.
- **sklearn.metrics.mean_squared_error**: Essa métrica é usada para avaliar o desempenho do modelo de regressão. Calcula o erro quadrático médio entre as previsões do modelo e os valores reais, indicando o quão próximo o modelo está dos dados reais.
- **yfinance**: Biblioteca para download de dados financeiros de ativos diretamente do Yahoo Finance. É útil para coletar dados de mercado de ações, permitindo realizar análises e prever tendências.

Essas bibliotecas foram escolhidas para fornecer uma base sólida na manipulação de dados, modelagem e avaliação, facilitando a implementação e análise do modelo de Bagging aplicado aos dados financeiros.


In [3]:
# Defina os tickers dos ativos e o período de 2 anos
tickers = ['AAPL', 'MSFT', 'GOOGL']
start_date = "2021-01-01"
end_date = "2023-01-01"

# Baixe os dados
data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')

# Salve os dados em um arquivo CSV
data.to_csv("ativos_yahoo_finance.csv")
print("Dados salvos em ativos_yahoo_finance.csv")

# Escolha um ativo para o modelo de previsão (exemplo: AAPL)
ticker = 'AAPL'

# Prepara os dados para o modelo - vamos usar o preço de fechamento ajustado
data_ticker = data[(ticker, 'Adj Close')].dropna()
data_ticker = data_ticker.to_frame(name='Adj Close')

# Crie uma coluna com o valor de fechamento do próximo dia para previsão
data_ticker['Target'] = data_ticker['Adj Close'].shift(-1)

# Remove valores nulos (última linha terá NaN no target)
data_ticker = data_ticker.dropna()

# Defina a janela para as variáveis de entrada
window_size = 90
for i in range(1, window_size + 1):
    data_ticker[f'Lag_{i}'] = data_ticker['Adj Close'].shift(i)

# Remova os valores nulos resultantes da lag
data_ticker = data_ticker.dropna()

# Divida os dados em variáveis de entrada (X) e variável alvo (y)
X = data_ticker.drop(columns=['Adj Close', 'Target'])
y = data_ticker['Target']

# Divida em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Crie o modelo Bagging com árvore de decisão como base
bagging_model = BaggingRegressor(
    base_estimator=DecisionTreeRegressor(),
    n_estimators=100,  # Número de amostras para o Bagging
    random_state=42
)

# Treine o modelo
bagging_model.fit(X_train, y_train)

# Faça previsões no conjunto de teste
y_pred = bagging_model.predict(X_test)

# Avalie o modelo usando Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE) no conjunto de teste: {mse}")

# Exibir algumas previsões em comparação com o valor real
predicoes_df = pd.DataFrame({'Real': y_test, 'Previsão': y_pred})
print(predicoes_df.head(10))


[*********************100%***********************]  3 of 3 completed


Dados salvos em ativos_yahoo_finance.csv
Mean Squared Error (MSE) no conjunto de teste: 27.96055887963235
                                 Real    Previsão
Date                                             
2022-09-01 00:00:00+00:00  153.933212  159.224198
2022-09-02 00:00:00+00:00  152.668625  158.012483
2022-09-06 00:00:00+00:00  154.081390  156.456094
2022-09-07 00:00:00+00:00  152.599487  156.262018
2022-09-08 00:00:00+00:00  155.474411  156.876643
2022-09-09 00:00:00+00:00  161.461426  150.964577
2022-09-12 00:00:00+00:00  151.986954  159.130143
2022-09-13 00:00:00+00:00  153.439224  158.798142
2022-09-14 00:00:00+00:00  150.534637  149.857278
2022-09-15 00:00:00+00:00  148.884766  155.911628


# Explicação do Código

Neste código, realizei uma série de etapas para aplicar o método de Bagging em um modelo de previsão de preços de ações:

1. **Download e Salvamento dos Dados**: Começei por definir os tickers e o período de dois anos para os ativos selecionados (AAPL, MSFT, GOOGL) e baixei os dados de preços históricos usando a biblioteca `yfinance`. Esses dados são salvos em um arquivo CSV para facilitar o acesso futuro.

2. **Preparação dos Dados**: Escolhi um ativo específico, neste caso, `AAPL`, e usei o preço de fechamento ajustado como a base para a previsão. Criei uma nova coluna `Target` contendo o preço de fechamento do próximo dia, que será nosso alvo de previsão.

3. **Criação de Lags**: Defini uma janela de 90 dias para capturar uma série temporal dos preços de fechamento anteriores. Para cada dia, criamos 90 variáveis de lag (`Lag_1`, `Lag_2`, ... `Lag_90`), que representam os preços de fechamento dos últimos 90 dias.

4. **Divisão de Dados**: Separando os dados em variáveis de entrada (`X`, que inclui as lags) e variável alvo (`y`, que é o preço do dia seguinte), dividi o conjunto de dados em conjuntos de treino e teste, mantendo a ordem temporal (sem `shuffle`).

5. **Construção do Modelo de Bagging**: Utilizei `BaggingRegressor` com `DecisionTreeRegressor` como o estimador base. Configuramos o `n_estimators` para 100, o que significa que o modelo treinará 100 árvores de decisão em diferentes amostras de dados, agregando os resultados para reduzir a variância.

6. **Treinamento e Previsão**: Treinei o modelo no conjunto de treino e fiz previsões no conjunto de teste.

7. **Avaliação**: Avaliei o desempenho do modelo usando o Mean Squared Error (MSE) para verificar a precisão das previsões. Por fim, exibi algumas previsões em comparação com os valores reais para entender o comportamento do modelo.

Essas etapas demonstram como implementar um modelo de Bagging para prever os preços de fechamento de ações, aumentando a robustez do modelo com base em múltiplas árvores de decisão.


# Análise dos Resultados

Os dados foram salvos no arquivo `ativos_yahoo_finance.csv` para facilitar o acesso posterior. O modelo de Bagging com árvore de decisão foi avaliado utilizando o Mean Squared Error (MSE) no conjunto de teste, resultando em um MSE de aproximadamente 27.96. Esse valor indica o erro médio quadrático entre as previsões do modelo e os valores reais, refletindo a precisão das previsões.

A tabela abaixo mostra uma amostra dos valores reais e das previsões do modelo para o preço de fechamento ajustado da ação `AAPL` nos primeiros dias do conjunto de teste. Observamos que, embora as previsões estejam próximas dos valores reais, há uma certa diferença em alguns dias, o que é esperado em modelos financeiros devido à alta volatilidade. O modelo captura tendências gerais, mas ainda apresenta variações que podem ser refinadas com ajustes no modelo ou nos dados. A comparação entre os valores reais e previstos permite identificar o desempenho do modelo ao longo do tempo.
